# Class 24: Linear regression and unsupervised learning

Plan for today:
- Linear regression
- Clustering


In [1]:
import YData

# YData.download.download_class_code(24)   # get class code    
# YData.download.download_class_code(24, TRUE) # get the code with the answers 

# YData.download.download_homework(9)  # downloads the homework 

# project review template
# YData.download.download_class_file('reviewer_template.ipynb', 'homework')


If you are using colabs, you should run the code below.

In [2]:
# !pip install https://github.com/emeyers/YData_package/tarball/master
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import statistics
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from urllib.request import urlopen

import matplotlib.pyplot as plt
%matplotlib inline

# Suppress ConvergenceWarning - please ignore this code 
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


## 0. Feature normalization

If you look at the features we have been using in our analyses, you will notice that they are on very different scales. This is quite problematic for a KNN classifier since the classifier is finding the distance between each data point, so features that have large values will dominate this distance. 

Let's explore the scales that different features have by looking at some descriptive statistics. In particular, let's go back to the manually created `X_train`, `X_test`, `y_train`, `y_test` to examine the scale that different features are measured on.


In [4]:
# Create the training and test splots of the data using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_penguin_features,  
                                                    y_penguin_labels, 
                                                    random_state = 0)

# Get summary statistics of the training data using the .describe() method
X_train.describe()

NameError: name 'train_test_split' is not defined

Let's do a z-score transformation of our features which set the mean of the features to 0 and the standard deviation to 1. We can do this using the using the `StandardScaler()` object as follows: 

1. Create a new `StandardScaler()` object using `scaler = StandardScaler()` 

2. Have the `scaler` object learn the means and standard deviations of our training data by calling the `scaler.fit(X)` function on the training data.

3. Use the fit `scaler` object to transform both the training and test features so that all features are on a similar scale by calling the `.transform(X)` method. 


In [ ]:
from sklearn.preprocessing import StandardScaler


# learning the mean and standard deviations to scale the features

scaler = StandardScaler()

scaler.fit(X_train)


In [ ]:
# z-score transform the features 

X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

type(X_test_transformed)

Let's now look at our transformed training data...

In [ ]:
# view descriptive statistics on the transformed features

X_train_transformed_df = pd.DataFrame(X_train_transformed, 
                                      columns = X_train.columns)

X_train_transformed_df.describe()

Let's see how our classification accuracy changes using the z-score transformed data

In [ ]:
# apply KNN classification on the normalized features

knn = KNeighborsClassifier(n_neighbors = 1) 
knn.fit(X_train_transformed, y_train)
knn.score(X_test_transformed, y_test)

In order to transform our features inside a cross-validation loop, we can set up a pipeline. This pipeline will do the following:

1. It will split the data into a training and test set
2. It will fit the transformation of the features on the training set (i.e., learn the means and standard deviations on the training set). 
3. It will apply a z-score transformation of the training and test set based on the features learned in step 2
4. It will train the classifier on the transformed data
5. It will measure the classification accuracy on the test data
6. It will repeat this process k times, where k here refers to how many cross-validation splits we are using

In order to do this in scikit-learn we can use a `Pipeline` object which sets up the stages of transformation and classification. We can then use the `cross_val_score()` function to run cross-validation on this pipeline. 

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import KFold


# create a pipeline for running cross-validation with feature normalization

# components that go into the pipeline
scaler = StandardScaler()
knn = KNeighborsClassifier(n_neighbors = 1) 

# build the pipeline
#pipeline = Pipeline([('transformer', scaler), ('estimator', knn)])
pipeline = make_pipeline(scaler, knn)

# get the cross-validation scores
scores = cross_val_score(pipeline, 
                         X_penguin_features, 
                         y_penguin_labels, 
                         cv = 5)


# print out the mean score over the 5 cross-validation splits
scores.mean()

## 1. Linear regression

In regression, we try to predict a quantitative variable y, from a set of features X. 

Let's explore this by predicting the body mass of penguins (in grams) from other quantitative features of a penguin (e.g., their bill and flipper sizes). 


In [ ]:
penguins = sns.load_dataset("penguins")

penguins = penguins.dropna()

penguins = penguins.sample(frac = 1)

penguins.head()

In [ ]:
# get the features and the labels

X_penguin_features = penguins[['bill_length_mm', 
                               'bill_depth_mm',
                               'flipper_length_mm']]

y_penguin = penguins['body_mass_g']


# also save the penguin species to use later
y_penguin_species = penguins['species']


Let's use scikit-learn to generate training and test data as we did previously for our KNN classifier. 

In [ ]:
from sklearn.model_selection import train_test_split

# split data into a training and test set

X_train, X_test, y_train, y_test = train_test_split(X_penguin_features,  
                                                    y_penguin, 
                                                    random_state = 0)

print(X_train.shape)
print(X_test.shape)

X_train.head(5)


We can now create a new linear regression model, fit it to data, and make predictions. The method names are again very similar to what we used for the KNN classifier (i.e., the `fit()` and predict()` methods). 

In [ ]:
from sklearn.linear_model import LinearRegression

# create a new linear regression modedl
linear_model = LinearRegression()

In [ ]:
# fit the model to our training data

linear_model.fit(X_train, y_train)

In [ ]:
# make predictions of the penguins body weight on the test data

body_mass_predictions = linear_model.predict(X_test)
body_mass_predictions[0:5]

We can assess the accuracy of our predictons using the root mean squared error which is defined as: 

$$RMSE = \sqrt{ \frac{1}{n}\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$$

Here $\hat{y}$ is the predictions made by our linear model on the test data (i.e., the predicted body weight) and y is the actual body weights for the points in our test set.

In [ ]:
# test the RMSE on the test data

RMSE = np.sqrt(np.mean((y_test - body_mass_predictions)**2))

RMSE   # shows (in grams) how for our predictions of penguin mass is typically off by


We can also use scikit-learn's `mean_squared_error()` to get the MSE, and we can use the `cross_val_score` to run k-fold cross-validation (again, in a very similar way to what we did for our KNN classifier). 

In [ ]:
from sklearn.metrics import mean_squared_error

# Use scikit-learn's mean_squared_error() function to get the RMSE
np.sqrt(mean_squared_error(y_test, body_mass_predictions))


In [ ]:
# using cross-validation
from sklearn.model_selection import cross_val_score

linear_model = LinearRegression()

scores = cross_val_score(linear_model, 
                         X_penguin_features,  
                         y_penguin, 
                         cv = 5, 
                         scoring='neg_mean_squared_error')

np.sqrt(np.mean(-1 * scores))

### Regression model equation

In linear regression, our predicted $\hat{y}$ values are given by the equation: $\hat{y} = b_0 + b_1 x_1 + ... + + b_k x_k$.

Let's fill out this equation for prediciting penguin body mass. 

To do this, let's start by extracting the intercept ($b_0$) and slope coefficients ($b_i's$) from our scikit-learn model.


In [ ]:

# fit the linear regression model to our training data
linear_model.fit(X_train, y_train)

# get the intercept and slope coefficients
sklearn_intercept = linear_model.intercept_
sklearn_coefficients = linear_model.coef_

# print out the coefficient values
(sklearn_intercept, sklearn_coefficients)  


Given these coefficient values can you write our the regression equation for predicting penguin body mass? 


#### Answer

$\hat{y}_{mass} = -6680.13 + 0.5049 \cdot x_{bill-length} + 21.6384 \cdot x_{bill-depth} +  52.1411 \cdot x_{lipper-length}$



#### Writing our own prediction function

Let's also write our own function called `get_predictions(b0_intercept, b_coefficients, X_data)` that takes the coefficient values and X values and returns predicted $\hat{y}$ values for each X value. In particular, the arguments to the function are:

1. `b0_intercept`: The linear regression intercept
2. `b_coefficients`: The linear regression slope coefficients
3. `X_data`: The X data values 

The returned value is a numpy ndarray of predictions for each X data point. 


In [ ]:
# write a function to get the predictions
def get_predictions(b0_intercept, b_coefficients, X_data):
    return np.sum(X_data.to_numpy() * b_coefficients, axis = 1) + b0_intercept 


# get the predicted values on the test data
predicted_vals = get_predictions(sklearn_intercept, sklearn_coefficients, X_test)


# see the it matches the scikit-learn predictions
predicted_vals_sklearn = linear_model.predict(X_test)
predicted_vals == predicted_vals_sklearn


## 1b. Inference on regression coefficients

We can also run inference procedures on our regression model using the statsmodel package. In particular, we can run hypothesis tests and create confidence intervals for our regression coefficents. 

When running a hypothesis test, our hypotheses are:

$H_0: \beta_i = 0$  
$H_A: \beta_i \ne 0$


In [ ]:
# Hypothesis test on regression coeffients - which coefficients are statistically significantly different from zero? 
# (and confidence interval)

import statsmodels.api as sm

# add a constant value of 1 to our data
X_train_with_constant = sm.add_constant(X_train) 

# fit the linear regression model using the OLS function
sm_linear_model = sm.OLS(y_train, X_train_with_constant).fit()

# get information on the regression coefficients found
print(sm_linear_model.summary())


## 2. Unsupervised learning: clustering

We can do k-means clustering in scikit-learn using the `KMeans()` object.


In [ ]:
from sklearn.cluster import KMeans

# fit k-means with 3 clusters 

kmeans = KMeans(n_clusters=3)
kmeans.fit(X_penguin_features)

In [ ]:
# see which cluster each point belongs to 

predicted_labels = kmeans.predict(X_penguin_features)
predicted_labels

In [ ]:
# look at a matrix of which penguin types end up in which cluster 

matrix = pd.DataFrame({'labels': predicted_labels, 
                       'species': y_penguin_species})

ct = pd.crosstab(matrix['labels'], matrix['species'])
print(ct)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# do clustering with feature normalization 
scaler = StandardScaler()
pipeline = make_pipeline(scaler, kmeans)

pipeline.fit(X_penguin_features)

In [ ]:
# see which cluster each (normalized) point belongs to

predicted_labels2 = pipeline.predict(X_penguin_features)

predicted_labels2


In [ ]:
# look at a matrix of which penguin types end up in which cluster 

matrix_new = pd.DataFrame({'labels': predicted_labels2, 
                           'species': y_penguin_species})

ct_new = pd.crosstab(matrix_new['labels'], 
                     matrix_new['species'])
print(ct_new)

### 2b. Unsupervised learning: Hierarchical clustering


In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy

#  Ward's method adds points to a cluster that minimizes the sum of squared differences within all clusters
clusters = hierarchy.linkage(X_penguin_features, method="ward")   


In [ ]:
# display a dendrogram
dendrogram = hierarchy.dendrogram(clusters)

In [ ]:
# cluster points into 3 clusters 
clustering_model = AgglomerativeClustering(n_clusters=3, linkage="ward")
clustering_model.fit(X_penguin_features)

# get the predicted cluster for each point
labels = clustering_model.labels_

labels

In [ ]:
# visualize how well the clustering matches the penguin species

sns.relplot(X_penguin_features, 
            x='bill_length_mm', 
            y='flipper_length_mm', 
            hue=labels, 
            style = y_penguin_species,
            palette="Set2");


In [ ]:
%%capture

# You can run this code to covert this Jupyter notebook into a pdf
!quarto render class_24_answers.ipynb --cache-refresh --to pdf 